# Importing the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [2]:
data=pd.read_html("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai")

In [3]:
df=pd.DataFrame(data[0])
df.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270


# Get coordinates

In [4]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
coords = [ get_latlng(area) for area in df["Area"].tolist() ]

In [6]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [7]:
df_coords.head()

,Latitude,Longitude
0,19.12906,72.84644
1,19.10836,72.86233
2,19.12505,72.83248
3,19.12632,72.82433
4,19.14316,72.82487


In [8]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [9]:
df.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.12906,72.84644
1,"Chakala, Andheri",Western Suburbs,19.10836,72.86233
2,D.N. Nagar,"Andheri,Western Suburbs",19.12505,72.83248
3,Four Bungalows,"Andheri,Western Suburbs",19.12632,72.82433
4,Lokhandwala,"Andheri,Western Suburbs",19.14316,72.82487


In [10]:
df=df.drop(['Location'],axis=1)
df.head()

,Area,Latitude,Longitude
0,Amboli,19.12906,72.84644
1,"Chakala, Andheri",19.10836,72.86233
2,D.N. Nagar,19.12505,72.83248
3,Four Bungalows,19.12632,72.82433
4,Lokhandwala,19.14316,72.82487


In [11]:
#df=df.drop(['Neighborhood'],axis=1)
df.head()

,Area,Latitude,Longitude
0,Amboli,19.12906,72.84644
1,"Chakala, Andheri",19.10836,72.86233
2,D.N. Nagar,19.12505,72.83248
3,Four Bungalows,19.12632,72.82433
4,Lokhandwala,19.14316,72.82487


In [12]:
df.rename(columns={'Area':'Neighborhood'},inplace=True)
df.head()

,Neighborhood,Latitude,Longitude
0,Amboli,19.12906,72.84644
1,"Chakala, Andheri",19.10836,72.86233
2,D.N. Nagar,19.12505,72.83248
3,Four Bungalows,19.12632,72.82433
4,Lokhandwala,19.14316,72.82487


In [13]:
df.shape

(93, 3)

# Visualizing the neighborhoods

In [14]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [15]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  

In [16]:
display(map_mum)

In [17]:
# define Foursquare Credentials and Version
CLIENT_ID = 'PI2XNUC4VTNOOG3FBMAOLSEA0O3OIAMOGB2C45N4HM4HFSQ2' # your Foursquare ID
CLIENT_SECRET = 'LF4DEG5TPMFUMJEUQWBY5VTV5QIXB0AQ52BXPTAFOLB1BFWF' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PI2XNUC4VTNOOG3FBMAOLSEA0O3OIAMOGB2C45N4HM4HFSQ2
CLIENT_SECRET:LF4DEG5TPMFUMJEUQWBY5VTV5QIXB0AQ52BXPTAFOLB1BFWF


# Finding the best locations within a particular radius 

In [18]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
    
    # make the GET request
    temp=requests.get(url).json()
    results = temp['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7583, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Amboli,19.12906,72.84644,Merwans Cake shop,19.119300,72.845418,Bakery
1,Amboli,19.12906,72.84644,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
2,Amboli,19.12906,72.84644,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Amboli,19.12906,72.84644,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Amboli,19.12906,72.84644,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant


In [20]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,74,74,74,74,74,74
Agripada,77,77,77,77,77,77
Altamount Road,90,90,90,90,90,90
Amboli,75,75,75,75,75,75
Amrut Nagar,92,92,92,92,92,92
Asalfa,100,100,100,100,100,100
Ballard Estate,100,100,100,100,100,100
Bandstand Promenade,100,100,100,100,100,100
Bangur Nagar,100,100,100,100,100,100


In [21]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Mughlai Restaurant', 'Falafel Restaurant',
       'Sandwich Place', 'American Restaurant', 'Pizza Place', 'Brewery',
       'Multiplex', 'Chinese Restaurant', 'Pub', 'Ice Cream Shop',
       'Indian Restaurant', 'BBQ Joint', 'Lounge', 'Diner', 'Coffee Shop',
       'Jewelry Store', 'Fast Food Restaurant', 'Gym / Fitness Center',
       'Bar', 'Dessert Shop', 'Residential Building (Apartment / Condo)',
       'College Gym', "Women's Store", 'Department Store', 'Café',
       'Sports Bar', 'Hotel', 'Athletics & Sports', 'Bus Station',
       'Camera Store', 'Shopping Mall', 'Pharmacy', 'Boutique',
       'Electronics Store', 'Accessories Store', 'Restaurant', 'Gym Pool',
       'Asian Restaurant', 'Airport Service', 'Italian Restaurant',
       'Seafood Restaurant', 'Snack Place', 'Spa', 'Bagel Shop',
       'Maharashtrian Restaurant', 'Juice Bar', 'Nightclub', 'Food Truck',
       'Airport Lounge'], dtype=object)

In [22]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [23]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7583, 218)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(93, 218)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Aarey Milk Colony,0.000000,0.000000,0.000000,0.00,0.00,0.013514,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.00,0.000000,0.000000,0.013514,0.000000,0.040541,0.000000,0.000000,0.00,0.000000,0.000000,0.013514,0.027027,0.000000,0.000000,0.000000,0.013514,0.013514,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.040541,0.000000,0.00,0.000000,0.013514,0.013514,0.000000,0.000000,0.040541,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.00,0.000000,0.013514,0.027027,0.00,0.013514,0.000000,0.000000,0.013514,0.000000,0.135135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.013514,0.013514,0.000000,0.0

In [25]:
len(kl_grouped[kl_grouped['Gym / Fitness Center'] > 0])


58

In [26]:
mum_gym = kl_grouped[["Neighborhoods","Gym / Fitness Center"]]

In [27]:
mum_gym.head()

,Neighborhoods,Gym / Fitness Center
0,Aarey Milk Colony,0.013514
1,Agripada,0.000000
2,Altamount Road,0.011111
3,Amboli,0.013333
4,Amrut Nagar,0.043478


In [28]:
kclusters = 4

mum_clustering = mum_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 1, 2, 0, 2, 3, 2])

In [29]:
mum_merged = mum_gym.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [30]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Gym / Fitness Center,Cluster Labels
0,Aarey Milk Colony,0.013514,0
1,Agripada,0.000000,2
2,Altamount Road,0.011111,0
3,Amboli,0.013333,0
4,Amrut Nagar,0.043478,1


In [31]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(93, 5)


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.013514,0,19.166150,72.859410
1,Agripada,0.000000,2,18.976280,72.826150
2,Altamount Road,0.011111,0,18.964338,72.807838
3,Amboli,0.013333,0,19.129060,72.846440
4,Amrut Nagar,0.043478,1,19.145160,72.846740


In [32]:
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(93, 5)


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.013514,0,19.166150,72.859410
33,Dhobitalao,0.010000,0,18.943956,72.829296
37,Four Bungalows,0.010000,0,19.126320,72.824330
39,Gowalia Tank,0.010101,0,18.964460,72.811170
44,Irla,0.010000,0,19.016390,72.829010
47,Kala Ghoda,0.010000,0,18.927240,72.832790
48,Kalina,0.010000,0,19.077680,72.864720
50,Kemps Corner,0.010000,0,18.964660,72.804530
54,Lower Parel,0.010000,0,18.998140,72.828120
55,Mahalaxmi,0.010000,0,18.946860,72.828890


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 0

In [34]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.013514,0,19.166150,72.859410
33,Dhobitalao,0.010000,0,18.943956,72.829296
37,Four Bungalows,0.010000,0,19.126320,72.824330
39,Gowalia Tank,0.010101,0,18.964460,72.811170
44,Irla,0.010000,0,19.016390,72.829010
47,Kala Ghoda,0.010000,0,18.927240,72.832790
48,Kalina,0.010000,0,19.077680,72.864720
50,Kemps Corner,0.010000,0,18.964660,72.804530
54,Lower Parel,0.010000,0,18.998140,72.828120
55,Mahalaxmi,0.010000,0,18.946860,72.828890


# Cluster 1

In [35]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
45,Jogeshwari West,0.050000,1,19.141160,72.843320
84,Thane,0.041667,1,19.140932,72.882604
34,Dindoshi,0.037037,1,19.168610,72.880360
4,Amrut Nagar,0.043478,1,19.145160,72.846740
53,Lokhandwala,0.040000,1,19.143160,72.824870


# Cluster 2

In [36]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
63,Marol,0.0,2,19.119050,72.882810
13,C.G.S. colony,0.0,2,19.138910,72.938170
65,Mira Road,0.0,2,19.265705,72.870693
90,Virar,0.0,2,19.016570,72.858530
67,Nahur,0.0,2,19.153650,72.946690
11,Bhuleshwar,0.0,2,18.951300,72.829300
69,Nalasopara,0.0,2,19.420000,72.814050
85,Uttan,0.0,2,26.866340,80.938840
71,Navy Nagar,0.0,2,18.906030,72.815450
87,Vidyavihar,0.0,2,19.023250,72.843900


# Cluster 3

In [37]:
mum_merged.loc[mum_merged['Cluster Labels'] == 3]


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
83,Thakur village,0.016129,3,19.210200,72.875410
86,Versova,0.020000,3,19.137690,72.813480
43,Indian Institute of Technology Bombay campus,0.022472,3,19.123840,72.911170
41,Hiranandani Gardens,0.024390,3,19.118970,72.906820
27,D.N. Nagar,0.020000,3,19.125050,72.832480
8,Bangur Nagar,0.020000,3,19.167850,72.832920
73,Pali Hill,0.020000,3,19.067720,72.825240
19,Chembur,0.023256,3,19.053995,72.899675
51,Khar Danda,0.020000,3,19.084250,72.826870
52,Kherwadi,0.020000,3,19.059520,72.845510


Observations:
Most of the fitness centers can be found in the clusters 0, 1 and 3 in Mumbai, with the highest number in cluster 1. Cluster 2 has no fitness centers in the neighborhoods. This represents a great opportunity and high potential areas to open new fitness centers. Meanwhile, cluster 1 is likely suffering from intense competition due to large number of fitness centers. Therefore, this project recommends capitalizing on these findings to open new fitness centers in neighborhoods of cluster 2, and with unique selling propositions, can even succeed in cluster 0 with moderate competition. Lastly, potential investors are advised to avoid neighborhoods in cluster 1 which already have high concentration of fitness centers and suffering from intense competition.